In [1]:
from ipywidgets import interact, widgets, Output
from statsmodels.stats.power import NormalIndPower
from IPython.display import display, clear_output
from plotly import graph_objects as go
import numpy as np

In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [3]:
# create an output widget
output = Output()

In [4]:
# create output for heatmap
heatmap_output = Output()

In [5]:
def generate_heatmap(budget, cpia, IC, alpha,effect_size, conversion_rate):
    #print(f"Generating heatmap with: budget={budget}, cpia={cpia}, IC={IC}, alpha={alpha}, effect_size={effect_size}, conversion_rate={conversion_rate}")  # Debug print
    budget_range = np.linspace(1_000, 100_000, 100)
    cpia_range = np.linspace(0.1, 2_000, 10)
    power_values = np.zeros((len(budget_range),len(cpia_range)))
    for i, b in enumerate(budget_range):
        for j,c in enumerate(cpia_range):
            power_values[i,j] = calculate_power(b, c, IC, alpha, effect_size, conversion_rate)
    
    fig = go.Figure(data = go.Heatmap(z = power_values,
                                     x = cpia_range,
                                     y = budget_range,
                                     colorscale = 'Viridis'))
    fig.update_layout(title = 'Power Analysis Heatmap',
                     xaxis_title = 'CPiA',
                     yaxis_title = 'Budget')
    with heatmap_output:
        clear_output(wait = True)
        fig.show()

In [6]:

def calculate_power(budget=25000, cpia=100, IC=0.06896019, alpha=0.1, effect_size=0.002291, conversion_rate=0.00185):
    incrementatals = budget/cpia
    expected_conversions = incrementatals/IC
    expected_sample = expected_conversions/conversion_rate
    power_analysis = NormalIndPower()

    power = power_analysis.solve_power(effect_size=effect_size,
                                       alpha=alpha,
                                       nobs1=expected_sample,
                                       ratio=1,
                                       alternative='two-sided')
    return power

In [7]:
# Function to update the output widget
def update_output(*args):
  power = calculate_power(budget.value, cpia.value, IC.value, alpha.value, effect_size.value, conversion_rate.value)
  output.clear_output()
  with output:
    print(f"Calculated Power: {power}")
  generate_heatmap(budget.value,cpia.value, IC.value, alpha.value, effect_size.value, conversion_rate.value)

In [8]:
# Initialize widgets
budget = widgets.IntSlider(value = 25_000,min = 1_000,max = 100_000, description = "Budget: ")

cpia = widgets.FloatSlider(value = 100, min = 0.01, max = 2_000, readout = True, description = "CPiA: ")

IC = widgets.BoundedFloatText(value = 0.06896019,min = 0, max = 1 , description = "IC: " )

alpha = widgets.BoundedFloatText(value = 0.1,min = 0, max = 1 , description = "Alpha: " )

effect_size = widgets.BoundedFloatText(value = 0.002291,min = 0, max = 20 , description = "Effect Size: " )

conversion_rate = widgets.BoundedFloatText(value = 0.00185,min = 0, max = 1 , description = "CVR: " )

In [9]:
# observes to vall fucntion when widget value changes
budget.observe(update_output,  'value')
cpia.observe(update_output, 'value')
IC.observe(update_output, 'value')
alpha.observe(update_output,  'value')
effect_size.observe(update_output,  'value')
conversion_rate.observe(update_output,  'value')

In [10]:
text_0 = widgets.HTML(value = "<h1>Power Analysis</h1>")
text_1 = widgets.HTML(value = "<h2> What is the planning budget?</h2>")
text_2 = widgets.HTML(value = "<h2> What is the CPiA?</h2>")
text_3 = widgets.HTML(value = "<h2> What is the IC(Incremetnal Contribution)?</h2>")
text_4 = widgets.HTML(value = "if your IC is in percentage form, eg: 4.5%, please put 0.045 in the below box")
text_5 = widgets.HTML(value = "<h2>What is the significance level AKA alpha?</h2>")
text_6 = widgets.HTML(value = "<h2>What is the Effect Size?</h2>")
text_7 = widgets.HTML(value = "<h2>What is the conversion rate?</h2>")
text_8 = widgets.HTML(value = "if your conversion rate is in percentage form, eg: 0.2%, then put 0.002 in the below box")
text_9 = widgets.HTML(value = "<h2>Based on the above info, the expected power is:</h2>")


In [11]:
# Layout
vbox_text = widgets.VBox([text_0, text_1, budget, text_2, cpia, text_3, 
                          text_4, IC, text_5, alpha, text_6, effect_size, text_7, text_8, 
                          conversion_rate, text_9, output,heatmap_output])

In [12]:
# Initial call to pupulate the output
update_output()

In [13]:
display(vbox_text)